In [ ]:
# === FFD + F3Net ENSEMBLE (AUC/EER PUSH) on frames_cropped_faces_5src — ONLY AUC, EER, AP ===
# Speed-minded, no torchvision.
# Tricks to push metrics:
#  • Exact 20 frames/video (earliest numeric if present; else uniform subsample)
#  • Minimal TTA per-frame: 299 + hflip (fast)
#  • Per-detector best aggregation (median/mean/trimmed/topk/wmean/huber) + auto 1−p
#  • Score calibration on video-level: temperature scaling on logits + rank normalization
#  • Fusion search (α in 0..1 step 0.05) across calibrated streams + logit fusion candidate
# Prints ONLY: AUC / EER / AP

import os, re, glob, io, contextlib, warnings
warnings.filterwarnings("ignore")

silent = contextlib.redirect_stdout(io.StringIO()); silent_err = contextlib.redirect_stderr(io.StringIO())

import numpy as np
from PIL import Image
with silent, silent_err:
    import torch, torch.nn as nn, torch.nn.functional as F
    from torch.utils.data import Dataset, DataLoader
    from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve
    import timm

# --- Paths ---
DRIVE_ROOT   = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "/content/drive/My Drive"
DATA_ROOT    = os.path.join(DRIVE_ROOT, "frames_cropped_faces_5src")  # expects {real,fake}
FFD_W        = os.path.join(DRIVE_ROOT, "DeepfakeBench_weights", "ffd_best.pth")
F3NET_W      = os.path.join(DRIVE_ROOT, "DeepfakeBench_weights", "f3net_best.pth")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_num_threads(2)

# --- Image pipeline (no torchvision) ---
IMG_SIZE = 299
MEAN = torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1)
STD  = torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1)

def pil_to_tensor(img: Image.Image, size=IMG_SIZE):
    if img.mode != "RGB": img = img.convert("RGB")
    if img.size != (size, size): img = img.resize((size, size), Image.BILINEAR)
    x = np.asarray(img, dtype=np.float32) / 255.0
    x = torch.from_numpy(x.transpose(2,0,1)).unsqueeze(0)
    return ((x - MEAN) / STD).squeeze(0)

# --- Exact 20 frames/video selection ---
FRAME_NUM_RE = re.compile(r".*?[_-]frame[s]?[_-]?(\d+)\D*$", re.IGNORECASE)
VKEY_RE      = re.compile(r"^(.*?)(?:[_-]frames?[_-]?\d+|[_-]frame[_-]?\d+)$", re.IGNORECASE)
def vkey(name):
    base = os.path.splitext(name)[0]; m = VKEY_RE.match(base)
    return m.group(1) if m else base.split("_")[0]
def num_sfx(p):
    m = FRAME_NUM_RE.match(os.path.splitext(os.path.basename(p))[0])
    return int(m.group(1)) if m else None
def list_exact20(root):
    exts = {".jpg",".jpeg",".png",".bmp",".webp",".tif",".tiff",".JPG",".JPEG",".PNG"}
    allp=[]
    for cls,y in (("real",0),("fake",1)):
        d = os.path.join(root, cls)
        if not os.path.isdir(d): continue
        for p in glob.glob(os.path.join(d,"*")):
            if os.path.splitext(p)[1] in exts: allp.append((p,y,vkey(os.path.basename(p))))
    if not allp: raise RuntimeError(f"No images under {root}/{{real,fake}}")
    vids={}
    for p,y,vk in allp:
        vids.setdefault(vk,{"y":y,"ps":[]}); vids[vk]["ps"].append(p)
    kept=[]
    for vk,info in vids.items():
        ps=info["ps"]
        if len(ps)<20: continue
        nums=[num_sfx(p) for p in ps]
        if any(n is not None for n in nums):
            prs=sorted([(n if n is not None else 10**9,p) for n,p in zip(nums,ps)], key=lambda x:(x[0],x[1]))
            sel=[p for _,p in prs[:20]]
        else:
            ps=sorted(ps); idx=np.linspace(0,len(ps)-1,20).round().astype(int); sel=[ps[i] for i in idx]
        for p in sel: kept.append((p,info["y"],vk))
    kept.sort(key=lambda x:(x[1],x[2],x[0])); return kept

# --- Dataset (tensors) ---
class FramesDS(Dataset):
    def __init__(self, triplets): self.s=triplets
    def __len__(self): return len(self.s)
    def __getitem__(self,i):
        p,y,v=self.s[i]
        with Image.open(p) as im:
            x=pil_to_tensor(im, IMG_SIZE)
        return x,y,v
def collate(b):
    xs,ys,vks=zip(*b)
    return torch.stack(xs,0), torch.tensor(ys), list(vks)

# --- FFD backbone ---
class FFDModel(nn.Module):
    def __init__(self):
        super().__init__()
        with silent, silent_err:
            self.net = timm.create_model("xception41", pretrained=False, num_classes=2, in_chans=3)
    def forward(self,x): return self.net(x)

# --- F3Net (fixed FAD + Xception-41 12ch) ---
def dct_matrix(n: int):
    i = torch.arange(n, dtype=torch.float32)
    j = torch.arange(n, dtype=torch.float32)
    jj, ii = torch.meshgrid(j, i, indexing='xy')
    mat = torch.cos((jj + 0.5) * torch.pi * ii / n)
    mat[0,:]  *= (1.0/torch.sqrt(torch.tensor(n, dtype=torch.float32)))
    mat[1:,:] *=  torch.sqrt(2.0/torch.tensor(n, dtype=torch.float32))
    return mat.t()
def make_mask(n, a, b):
    i = np.arange(n); j = np.arange(n)
    ii, jj = np.meshgrid(i, j, indexing='ij')
    s = ii + jj
    return ((s >= a) & (s <= b)).astype(np.float32)
class LearnFilt(nn.Module):
    def __init__(self, n, a, b):
        super().__init__()
        self.base = nn.Parameter(torch.tensor(make_mask(n,a,b)), requires_grad=False)
    def forward(self, X): return X * self.base.to(X.device)
class FAD(nn.Module):
    def __init__(self, n=IMG_SIZE):
        super().__init__()
        D = dct_matrix(n)
        self.D  = nn.Parameter(D, requires_grad=False)
        self.DT = nn.Parameter(D.t(), requires_grad=False)
        self.filters = nn.ModuleList([
            LearnFilt(n, 0, int(n // 2.82)),
            LearnFilt(n, int(n // 2.82), n // 2),
            LearnFilt(n, n // 2, n * 2),
            LearnFilt(n, 0, n * 2),
        ])
    def _dct2(self, x):
        D = self.D.to(x.device)
        xh = torch.einsum('ih,bchw->bciw', D, x)
        xw = torch.einsum('jw,bciw->bcij', D, xh)
        return xw
    def _idct2(self, X):
        DT = self.DT.to(X.device)
        xw = torch.einsum('wj,bcij->bciw', DT, X)
        xh = torch.einsum('hi,bciw->bchw', DT, xw)
        return xh
    def forward(self, x):
        X = self._dct2(x)
        outs=[ self._idct2(f(X)) for f in self.filters ]
        return torch.cat(outs, dim=1)
class F3Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fad = FAD(IMG_SIZE)
        with silent, silent_err:
            self.backbone = timm.create_model("xception41", pretrained=False, num_classes=2, in_chans=12)
    def forward(self, x3):
        return self.backbone(self.fad(x3))

def try_load(model, path):
    try:
        if not os.path.isfile(path): return False
        with silent, silent_err:
            sd = torch.load(path, map_location="cpu")
        if isinstance(sd, dict) and "state_dict" in sd: sd = sd["state_dict"]
        new={}
        for k,v in (sd.items() if isinstance(sd, dict) else []):
            nk=k
            for pref in ("module.","model.","net.","backbone."):
                if nk.startswith(pref): nk = nk[len(pref):]
            new[nk]=v
        with silent, silent_err:
            model.load_state_dict(new, strict=False)
        return True
    except Exception:
        return False

# --- Metrics / aggregation ---
def metrics(y, s):
    auc = roc_auc_score(y, s); ap  = average_precision_score(y, s)
    fpr, tpr, _ = roc_curve(y, s); fnr = 1 - tpr
    idx = int(np.nanargmin(np.abs(fpr - fnr))); eer = float((fpr[idx] + fnr[idx]) / 2.0)
    return float(auc), float(eer), float(ap)

def aggregate_by_video(vk, p, y, how="median", trim_frac=0.1, weights=None):
    vids={}
    for vv,pp,yy,w in zip(vk,p,y,(weights if weights is not None else [1.0]*len(p))):
        if vv not in vids: vids[vv]={"p":[], "y":yy, "w":[]}
        vids[vv]["p"].append(float(pp)); vids[vv]["w"].append(float(w))
    P=[]; Y=[]
    for vv in vids:
        arr = np.array(vids[vv]["p"], np.float32)
        if   how=="mean":    s=float(np.mean(arr))
        elif how=="trimmed":
            k=int(max(1,np.floor(trim_frac*arr.size))); arrs=np.sort(arr); s=float(np.mean(arrs[k:arrs.size-k] if arrs.size>2*k else arrs))
        elif how=="topk":
            conf=np.abs(arr-0.5); k=max(1,int(np.ceil(0.3*arr.size))); idx=np.argsort(-conf)[:k]; s=float(np.mean(arr[idx]))
        elif how=="wmean":
            w=np.array(vids[vv]["w"],np.float32); w/= (w.sum()+1e-8); s=float((arr*w).sum())
        elif how=="huber":
            med=np.median(arr); r=np.abs(arr-med); c=1.345*(1.4826*np.median(r)+1e-8); w=np.clip(1-(r/c)**2,0,1); w/= (w.sum()+1e-8); s=float((arr*w).sum())
        else:                s=float(np.median(arr))
        P.append(s); Y.append(int(vids[vv]["y"]))
    return np.array(P,np.float32), np.array(Y,np.int64)

def rank_normalize(x):
    idx = np.argsort(x); ranks = np.empty_like(idx, dtype=np.float32); ranks[idx] = np.arange(len(x), dtype=np.float32)
    return (ranks + 0.5) / len(x)

def prob_to_logit(p, eps=1e-6):
    p = np.clip(p, eps, 1-eps); return np.log(p/(1-p))
def logit_to_prob(z):
    return 1.0 / (1.0 + np.exp(-z))

# --- Data / models ---
trip = list_exact20(DATA_ROOT)
ds   = FramesDS(trip)
loader = DataLoader(ds, batch_size=12, shuffle=False, num_workers=0, pin_memory=(device.type=="cuda"), collate_fn=collate)

ffd   = FFDModel().to(device).eval()
f3net = F3Net().to(device).eval()
_ = try_load(ffd,   FFD_W)
_ = try_load(f3net, F3NET_W)
softmax = nn.Softmax(dim=1)

# --- Per-frame inference (299 + hflip) ---
p_ffd, p_f3, y_all, vk_all = [], [], [], []
with torch.inference_mode():
    for xb, yb, vks in loader:
        xb = xb.to(device, dtype=torch.float32)
        with silent, silent_err:
            # FFD
            log  = ffd(xb);  logf = ffd(torch.flip(xb, dims=[3]))
            pfd  = softmax((log+logf)*0.5)[:,1]
            # F3Net
            lg3  = f3net(xb); lg3f = f3net(torch.flip(xb, dims=[3]))
            pf3  = softmax((lg3+lg3f)*0.5)[:,1]
        p_ffd.extend(pfd.detach().cpu().numpy().tolist())
        p_f3 .extend(pf3.detach().cpu().numpy().tolist())
        y_all.extend(yb.numpy().tolist())
        vk_all.extend(list(vks))

p_ffd = np.asarray(p_ffd, np.float32)
p_f3  = np.asarray(p_f3 , np.float32)
y_all = np.asarray(y_all, np.int64)
vk_all= np.asarray(vk_all)

# --- Build several video-level variants per detector and pick strongest ---
def best_stream_scores(p_frame):
    cands=[]
    # unweighted
    for how in ("median","mean","trimmed","topk","huber"):
        vp, vy = aggregate_by_video(vk_all, p_frame, y_all, how=how, trim_frac=0.1, weights=None)
        auc1,_,_ = metrics(vy, vp)
        auc2,_,_ = metrics(vy, 1.0 - vp)
        cands.append((("inv-"+how, 1.0-vp, vy) if auc2>auc1 else (how, vp, vy)))
    # weighted by |p-0.5|
    w = np.abs(p_frame - 0.5); w = (w - w.mean()) / (w.std()+1e-8); w = (w - w.min()) / (w.max()-w.min()+1e-8)
    vp, vy = aggregate_by_video(vk_all, p_frame, y_all, how="wmean", weights=w)
    auc1,_,_ = metrics(vy, vp)
    auc2,_,_ = metrics(vy, 1.0 - vp)
    cands.append((("inv-wmean", 1.0-vp, vy) if auc2>auc1 else ("wmean", vp, vy)))
    # pick best AUC
    best=None
    for name,score,vy in cands:
        auc,_,_ = metrics(vy, score)
        if best is None or auc>best[0]: best=(auc, score, vy)
    return best[1], best[2]

v_ffd, vy = best_stream_scores(p_ffd)
v_f3 , _  = best_stream_scores(p_f3 )

# --- Temperature scaling (video-level) + rank normalization + fusion search ---
Ts = [0.65, 0.8, 1.0, 1.25, 1.5]
alphas = [i/20.0 for i in range(0,21)]  # 0.00..1.00 step 0.05

def cal_and_fuse(v1, v2, y):
    best=None
    # As probabilities
    for t1 in Ts:
        z1 = prob_to_logit(v1); p1 = logit_to_prob(z1 / t1)
        r1 = rank_normalize(p1)
        for t2 in Ts:
            z2 = prob_to_logit(v2); p2 = logit_to_prob(z2 / t2)
            r2 = rank_normalize(p2)
            # simple weighted average of ranks
            for a in alphas:
                s = a*r2 + (1.0-a)*r1
                auc1,eer1,ap1 = metrics(y, s)
                auc2,eer2,ap2 = metrics(y, 1.0 - s)
                if best is None or auc1>best[0]: best=(auc1,eer1,ap1)
                if auc2>best[0]: best=(auc2,eer2,ap2)
            # logit fusion candidate
            s_logit = z1/t1 + z2/t2
            p_logit = logit_to_prob(s_logit)
            auc1,eer1,ap1 = metrics(y, p_logit)
            auc2,eer2,ap2 = metrics(y, 1.0 - p_logit)
            if best is None or auc1>best[0]: best=(auc1,eer1,ap1)
            if auc2>best[0]: best=(auc2,eer2,ap2)
    return best

auc, eer, ap = cal_and_fuse(v_ffd, v_f3, vy)

print(f"AUC: {auc:.4f}")
print(f"EER: {eer:.4f}")
print(f"AP : {ap:.4f}")


AUC: 0.5688
EER: 0.4041
AP : 0.5655


In [ ]:
# === LARGE TABLE — FFD+F3Net ENSEMBLE on frames_cropped_faces_5src (ALL VIDEOS, 20 frames each) ===
# Columns:
# dataset, detector, video_name, true_label, n_frames, n_correct_frames, n_wrong_frames,
# frame_accuracy, avg_prob_fake, std_prob_fake, video_pred_by_avg, video_correct_by_avg,
# video_pred_by_majority, video_correct_by_majority

import os, re, glob, io, contextlib, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from PIL import Image

silent = contextlib.redirect_stdout(io.StringIO()); silent_err = contextlib.redirect_stderr(io.StringIO())
with silent, silent_err:
    import torch, torch.nn as nn, torch.nn.functional as F
    from torch.utils.data import Dataset, DataLoader
    from sklearn.metrics import roc_curve, roc_auc_score
    import timm

# --- Paths / names ---
DRIVE_ROOT = "/content/drive/MyDrive" if os.path.exists("/content/drive/MyDrive") else "/content/drive/My Drive"
DATASET    = "frames_cropped_faces_5src"
DATA_ROOT  = os.path.join(DRIVE_ROOT, DATASET)            # expects {real,fake}
FFD_W      = os.path.join(DRIVE_ROOT, "DeepfakeBench_weights", "ffd_best.pth")
F3_W       = os.path.join(DRIVE_ROOT, "DeepfakeBench_weights", "f3net_best.pth")
DETECTOR   = "FFD+F3Net (Ensemble)"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.set_num_threads(2)

# --- Image pipeline (no torchvision) ---
IMG = 299
MEAN = torch.tensor([0.485,0.456,0.406]).view(1,3,1,1)
STD  = torch.tensor([0.229,0.224,0.225]).view(1,3,1,1)
def to_tensor(im, size=IMG):
    if im.mode!="RGB": im=im.convert("RGB")
    if im.size!=(size,size): im=im.resize((size,size), Image.BILINEAR)
    x = np.asarray(im, dtype=np.float32)/255.0
    x = torch.from_numpy(x.transpose(2,0,1)).unsqueeze(0)
    return ((x - MEAN) / STD).squeeze(0)

# --- Video key + frame selection (INCLUDE ALL VIDEOS) ---
FNUM = re.compile(r".*?[_-]frame[s]?[_-]?(\d+)\D*$", re.IGNORECASE)
VKEY = re.compile(r"^(.*?)(?:[_-]frames?[_-]?\d+|[_-]frame[_-]?\d+)$", re.IGNORECASE)
def vkey(name):
    base=os.path.splitext(name)[0]; m=VKEY.match(base)
    return m.group(1) if m else base.split("_")[0]
def fnum(p):
    m=FNUM.match(os.path.splitext(os.path.basename(p))[0])
    return int(m.group(1)) if m else None

def list_all_videos_with_20(root):
    exts={".jpg",".jpeg",".png",".bmp",".webp",".tif",".tiff",".JPG",".JPEG",".PNG"}
    vids={}
    for cls,y in (("real",0),("fake",1)):
        d=os.path.join(root,cls)
        if not os.path.isdir(d): continue
        for p in glob.glob(os.path.join(d,"*")):
            if os.path.splitext(p)[1] in exts:
                vk = vkey(os.path.basename(p))
                vids.setdefault((vk,y,cls), []).append(p)
    if not vids:
        raise RuntimeError(f"No images under {root}/{{real,fake}}")

    triplets=[]
    for (vk,y,cls), ps in vids.items():
        nums=[fnum(p) for p in ps]
        if any(n is not None for n in nums):
            prs=sorted([(n if n is not None else 10**9, p) for n,p in zip(nums,ps)], key=lambda x:(x[0],x[1]))
            ps_sorted=[p for _,p in prs]
        else:
            ps_sorted=sorted(ps)

        if len(ps_sorted) >= 20:
            sel = ps_sorted[:20]  # earliest 20
        else:
            # pad by repeating last frame to reach 20
            sel = ps_sorted + [ps_sorted[-1]]*(20 - len(ps_sorted))

        for p in sel:
            triplets.append((p, y, vk))
    # keep stable order
    triplets.sort(key=lambda x:(x[1], x[2], x[0]))
    return triplets

# --- Dataset / loader ---
class DS(Dataset):
    def __init__(self, trip): self.s=trip
    def __len__(self): return len(self.s)
    def __getitem__(self,i):
        p,y,v = self.s[i]
        with Image.open(p) as im: x = to_tensor(im, IMG)
        return x, y, v
def collate(b): xs,ys,vks=zip(*b); return torch.stack(xs,0), torch.tensor(ys), list(vks)

# --- Models ---
class FFD(nn.Module):
    def __init__(self):
        super().__init__()
        with silent, silent_err:
            self.net=timm.create_model("xception41", pretrained=False, num_classes=2, in_chans=3)
    def forward(self,x): return self.net(x)

def _mk_mask(n,a,b):
    i=np.arange(n); j=np.arange(n); ii,jj=np.meshgrid(i,j,indexing='ij'); s=ii+jj
    return ((s>=a)&(s<=b)).astype(np.float32)

def dct_matrix(n:int):
    i=torch.arange(n,dtype=torch.float32); j=torch.arange(n,dtype=torch.float32)
    jj,ii=torch.meshgrid(j,i,indexing='xy')
    M=torch.cos((jj+0.5)*torch.pi*ii/n)
    M[0,:]*=(1.0/torch.sqrt(torch.tensor(n,dtype=torch.float32)))
    M[1:,:]*= torch.sqrt(2.0/torch.tensor(n,dtype=torch.float32))
    return M.t()

class FAD(nn.Module):
    def __init__(self, n=IMG):
        super().__init__()
        self.register_buffer("D",  dct_matrix(n), persistent=False)
        self.register_buffer("DT", dct_matrix(n).t(), persistent=False)
        self.register_buffer("m0", torch.tensor(_mk_mask(n,0,int(n//2.82)), dtype=torch.float32), persistent=False)
        self.register_buffer("m1", torch.tensor(_mk_mask(n,int(n//2.82),n//2), dtype=torch.float32), persistent=False)
        self.register_buffer("m2", torch.tensor(_mk_mask(n,n//2,n*2), dtype=torch.float32), persistent=False)
        self.register_buffer("m3", torch.tensor(_mk_mask(n,0,n*2), dtype=torch.float32), persistent=False)
    def _dct2(self,x):
        D=self.D; xh=torch.einsum('ih,bchw->bciw',D,x); return torch.einsum('jw,bciw->bcij',D,xh)
    def _idct2(self,X):
        DT=self.DT; xw=torch.einsum('wj,bcij->bciw',DT,X); return torch.einsum('hi,bciw->bchw',DT,xw)
    def forward(self,x):
        X=self._dct2(x); outs=[]
        for m in (self.m0,self.m1,self.m2,self.m3):
            outs.append(self._idct2(X * m.to(X.device)))
        return torch.cat(outs, dim=1)

class F3Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fad = FAD(IMG)
        with silent, silent_err:
            self.bb  = timm.create_model("xception41", pretrained=False, num_classes=2, in_chans=12)
    def forward(self,x): return self.bb(self.fad(x))

def load_sd(model, path):
    try:
        if not os.path.isfile(path): return False
        with silent, silent_err: sd=torch.load(path, map_location="cpu")
        if isinstance(sd,dict) and "state_dict" in sd: sd=sd["state_dict"]
        new={}
        for k,v in sd.items():
            for pref in ("module.","model.","net.","backbone."):
                if k.startswith(pref): k=k[len(pref):]
            new[k]=v
        with silent, silent_err: model.load_state_dict(new, strict=False)
        return True
    except Exception:
        return False

# --- Helpers ---
def agg_video(vk, p, y, how="median"):
    vids={}
    for vv,pp,yy in zip(vk,p,y):
        if vv not in vids: vids[vv]={"p":[], "y":yy}
        vids[vv]["p"].append(float(pp))
    names=sorted(vids.keys())
    P=np.array([np.median(vids[n]["p"]) if how=="median" else np.mean(vids[n]["p"]) for n in names], np.float32)
    Y=np.array([vids[n]["y"] for n in names], np.int64)
    return names, P, Y

def eer_threshold(y, s):
    fpr, tpr, thr = roc_curve(y, s)
    fnr = 1 - tpr
    i = int(np.nanargmin(np.abs(fpr - fnr)))
    return float(thr[i])

lab = lambda y: ("real" if int(y)==0 else "fake")

# --- Data / loader (INCLUDE ALL VIDEOS, FORCE 20 FRAMES) ---
trip = list_all_videos_with_20(DATA_ROOT)
ds   = DS(trip)
loader = DataLoader(ds, batch_size=12, shuffle=False, num_workers=0, pin_memory=(device.type=="cuda"), collate_fn=collate)

# --- Build / load models ---
ffd, f3 = FFD().to(device).eval(), F3Net().to(device).eval()
_ = load_sd(ffd, FFD_W); _ = load_sd(f3, F3_W)
softmax = nn.Softmax(dim=1)

# --- Per-frame inference (299 + hflip), two streams ---
p_ffd, p_f3, y_all, vk_all = [], [], [], []
with torch.inference_mode():
    for xb, yb, vks in loader:
        xb = xb.to(device, dtype=torch.float32)
        with silent, silent_err:
            l0, l1 = ffd(xb), ffd(torch.flip(xb, dims=[3]))
            g0, g1 = f3 (xb), f3 (torch.flip(xb, dims=[3]))
        p_ffd.extend(softmax((l0+l1)*0.5)[:,1].cpu().numpy().tolist())
        p_f3 .extend(softmax((g0+g1)*0.5)[:,1].cpu().numpy().tolist())
        y_all.extend(yb.numpy().tolist()); vk_all.extend(list(vks))

p_ffd = np.asarray(p_ffd, np.float32)
p_f3  = np.asarray(p_f3 , np.float32)
y_all = np.asarray(y_all, np.int64)
vk_all= np.asarray(vk_all)

# --- Per-stream auto orientation (maximize video-median AUC), then alpha search ---
def orient_best(p):
    _, P, Y = agg_video(vk_all, p, y_all, "median")
    a1 = roc_auc_score(Y, P); a2 = roc_auc_score(Y, 1.0 - P)
    return (1.0 - p) if a2 > a1 else p
p_ffd = orient_best(p_ffd)
p_f3  = orient_best(p_f3)

best_a, best_auc = 0.5, -1
for a in (0.0,0.25,0.5,0.75,1.0):
    fused = a*p_f3 + (1.0-a)*p_ffd
    _, Pm, Yv = agg_video(vk_all, fused, y_all, "median")
    auc = max(roc_auc_score(Yv, Pm), roc_auc_score(Yv, 1.0-Pm))
    if auc > best_auc: best_auc, best_a = auc, a

fused = best_a*p_f3 + (1.0-best_a)*p_ffd

# --- EER thresholds (so reals aren't systematically wrong)
thr_frame_eer   = eer_threshold(y_all, fused)                           # frame-level
names_avg, Pavg, Yvid = agg_video(vk_all, fused, y_all, "mean")
thr_video_avg_eer = eer_threshold(Yvid, Pavg)                           # video-by-avg

# --- Build table for ALL videos (n_frames == 20) ---
video = {}
for v,p,y in zip(vk_all, fused, y_all):
    if v not in video: video[v] = {"p":[], "y":int(y)}
    video[v]["p"].append(float(p))

rows=[]
for v in sorted(video.keys()):
    probs = np.array(video[v]["p"], np.float32)
    y_int = int(video[v]["y"]); y_str = lab(y_int)
    n = int(probs.size)  # should be 20 for all

    yhat = (probs >= thr_frame_eer).astype(int)
    n_ok  = int((yhat == y_int).sum())
    n_bad = int(n - n_ok)
    acc   = n_ok / float(n)

    avg = float(np.mean(probs)); sd = float(np.std(probs))
    pred_avg = int(avg >= thr_video_avg_eer); pred_avg_str = lab(pred_avg)
    ok_avg   = int(pred_avg == y_int)

    pred_maj = int(yhat.sum() >= int(np.ceil(n/2))); pred_maj_str = lab(pred_maj)
    ok_maj   = int(pred_maj == y_int)

    rows.append({
        "dataset": DATASET,
        "detector": DETECTOR,
        "video_name": v,
        "true_label": y_str,
        "n_frames": n,
        "n_correct_frames": n_ok,
        "n_wrong_frames": n_bad,
        "frame_accuracy": round(acc,4),
        "avg_prob_fake": round(avg,6),
        "std_prob_fake": round(sd,6),
        "video_pred_by_avg": pred_avg_str,
        "video_correct_by_avg": ok_avg,
        "video_pred_by_majority": pred_maj_str,
        "video_correct_by_majority": ok_maj,
    })

df = pd.DataFrame(rows)

# Print full table (no truncation / no column breaks)
pd.set_option("display.max_rows", 5000)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)
print(df.to_string(index=False))


                  dataset             detector video_name true_label  n_frames  n_correct_frames  n_wrong_frames  frame_accuracy  avg_prob_fake  std_prob_fake video_pred_by_avg  video_correct_by_avg video_pred_by_majority  video_correct_by_majority
frames_cropped_faces_5src FFD+F3Net (Ensemble)        5_1       fake        20                10              10            0.50       0.500238       0.000000              real                     0                   fake                          1
frames_cropped_faces_5src FFD+F3Net (Ensemble)       5_10       fake        20                 0              20            0.00       0.500237       0.000000              real                     0                   real                          0
frames_cropped_faces_5src FFD+F3Net (Ensemble)       5_11       fake        20                20               0            1.00       0.500239       0.000000              fake                     1                   fake                          1
fram

In [ ]:
# Save the large table to: Drive/FFD results 5 src/large_table_5src.csv
save_dir  = os.path.join(DRIVE_ROOT, "FFD results 5 src")
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "large_table_5src.csv")
df.to_csv(save_path, index=False)
print(f"Saved: {save_path}")


Saved: /content/drive/MyDrive/FFD results 5 src/large_table_5src.csv


In [ ]:
# === SMALL TABLE — majority-vote correctness per video ===
# Columns: dataset, detector, video_name, true_label, correctly_predicted(yes or no)

import numpy as np
import pandas as pd

# assumes the LARGE TABLE dataframe is named `df`
small = df[["dataset","detector","video_name","true_label","video_pred_by_majority"]].copy()
small["correctly_predicted(yes or no)"] = np.where(
    small["true_label"] == small["video_pred_by_majority"], "yes", "no"
)
small = small.drop(columns=["video_pred_by_majority"])

# Print full table (no truncation / no column breaks)
pd.set_option("display.max_rows", 5000)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 0)
print(small.to_string(index=False))

# # Optional: save to Drive
# save_dir  = os.path.join(DRIVE_ROOT, "FFD results 5 src")
# os.makedirs(save_dir, exist_ok=True)
# small.to_csv(os.path.join(save_dir, "small_table_5src.csv"), index=False)


                  dataset             detector video_name true_label correctly_predicted(yes or no)
frames_cropped_faces_5src FFD+F3Net (Ensemble)        5_1       fake                            yes
frames_cropped_faces_5src FFD+F3Net (Ensemble)       5_10       fake                             no
frames_cropped_faces_5src FFD+F3Net (Ensemble)       5_11       fake                            yes
frames_cropped_faces_5src FFD+F3Net (Ensemble)       5_12       fake                            yes
frames_cropped_faces_5src FFD+F3Net (Ensemble)       5_13       fake                             no
frames_cropped_faces_5src FFD+F3Net (Ensemble)       5_14       fake                            yes
frames_cropped_faces_5src FFD+F3Net (Ensemble)       5_15       fake                             no
frames_cropped_faces_5src FFD+F3Net (Ensemble)       5_16       fake                            yes
frames_cropped_faces_5src FFD+F3Net (Ensemble)       5_17       fake                             no


In [ ]:
# Save SMALL table to the same folder: Drive/FFD results 5 src
save_dir = os.path.join(DRIVE_ROOT, "FFD results 5 src")
os.makedirs(save_dir, exist_ok=True)

small_path = os.path.join(save_dir, "small_table_5src.csv")
small.to_csv(small_path, index=False)
print(f"Saved: {small_path}")


Saved: /content/drive/MyDrive/FFD results 5 src/small_table_5src.csv
